In [8]:
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing
import json
import tqdm
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from pathlib import Path

In [62]:
# send a HTTP GET request and store the response
gender_occupations = {"male": [
    "police officer",
    "electrician",
    "carpenter",
    "software developer"],
"female": [
    "social worker",
    "maid",
    "secretary",
    "nurse"],
"general": [
    "professor",
    "photographer",
    "artist",
    "manager"]
}
N_IMGS = 200

url = 'https://knn5.laion.ai/knn-service'
occs = [o for occupations in gender_occupations.values() for o in occupations]
with ThreadPoolExecutor(max_workers = 10) as executor:
    for occ in tqdm.tqdm(occs):
        data = {"text":f"A picture of a {occ}","image":None,"image_url":None,"embedding_input":None,"modality":"image","num_images":N_IMGS*4,"indice_name":"laion5B","num_result_ids":3000,"use_mclip":False,"deduplicate":True,"use_safety_model":False,"use_violence_detector":False,"aesthetic_score":"","aesthetic_weight":"0.5"}
        response = requests.post(url, data=json.dumps(data))
        response.raise_for_status()
        dicts = json.loads(response.text)

        urls = [(i, dic['url']) for i, dic in enumerate(dicts) if 'url' in dic]
        
        def func(i_url):
            i, url = i_url
            try: 
                r = requests.get(url)
                r.raise_for_status()
            except: 
                print(r, end=' ')
                return
            return r.content
        result = list(filter(bool, executor.map(func, urls)))
        assert len(result) >= 200

        def write(i_occ_content):
            i, occ, content = i_occ_content
            with open(f'laion_{occ}_{i}.jpg') as f:
                f.write(content)
        executor.map(write, [(i, occ, c) for i, c in enumerate(result)])

  0%|          | 0/12 [00:00<?, ?it/s]

<Response [404]> <Response [404]> <Response [403]> <Response [404]> <Response [404]> <Response [503]> <Response [404]> <Response [403]> 

  0%|          | 0/12 [00:17<?, ?it/s]


UnboundLocalError: local variable 'r' referenced before assignment